In [1]:
#!pip install tf-slim

In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from tf_slim.layers import layers as _layers

C:\Users\user\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\user\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\user\anaconda3\lib\site-packages\numpy\.libs\libopenblas.wcdjnk7yvmpzq2me2zzhjjrj3jikndb7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
nb_classes = 10

###### reshape를 통한 vector로 변환 

In [4]:
input_shape = X_train.shape[1] * X_train.shape[2]
X_train = X_train.reshape(-1, input_shape) 
y_train = to_categorical(y_train, nb_classes)

###### array to tensorflow

In [5]:
X_train = tf.image.convert_image_dtype(X_train, tf.float32)
y_train = tf.image.convert_image_dtype(y_train, tf.float32)

In [6]:
keep_prob = 0.7
learning_rate = 0.01
training_epochs = 5
batch_size = 100

In [7]:
X = tf.placeholder(tf.float32, [None, 784])
X_img = tf.reshape(X, [-1, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])

In [8]:
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))
L1 = tf.nn.conv2d(X_img, W1, strides=[1,1,1,1], padding="SAME")
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides = [1,2,2,1], padding="SAME")
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding="SAME")
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides = [1,2,2,1], padding="SAME")
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

In [10]:
W3 = tf.Variable(tf.random_normal([3,3,64,128], stddev=0.01))
L3 = tf.nn.conv2d(L2, W3, strides=[1,1,1,1], padding="SAME")
L3 = tf.nn.relu(L3)
L3 = tf.nn.max_pool(L3, ksize=[1,2,2,1], strides = [1,2,2,1], padding="SAME")
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)
L3 = tf.reshape(L3, [-1, 128 * 4 * 4])

# Fully connected layer 구간

In [11]:
W4 = tf.get_variable("W4", shape=[128 * 4 * 4, 625], initializer= tf.compat.v1.keras.initializers.glorot_normal)
b4 = tf.Variable(tf.random_normal([625]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)

In [12]:
W5 = tf.get_variable("W5", shape=[625, 10], initializer=tf.compat.v1.keras.initializers.glorot_normal)
b5 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L4, W5) + b5

In [13]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [14]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

# epoch 돌리는 부분

In [15]:
print("Learning started. It takes sometime.")
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(60000 / batch_size)
    for i in tqdm(range(total_batch)):
        batch_xs, batch_ys = X_train[(i*batch_size):batch_size+(i*batch_size)].eval(), y_train[(i*batch_size):batch_size+(i*batch_size)].eval()
        c, _ = sess.run([cost, optimizer], feed_dict={X: batch_xs, Y:batch_ys})
        avg_cost += c / total_batch
    print("Epoch:", "%04d" % (epoch + 1), "cost =", "{:.9f}".format(avg_cost))
print("Learning Finished!")

Learning started. It takes sometime.


100%|██████████| 600/600 [03:08<00:00,  3.18it/s]


Epoch: 0001 cost = 0.532569520


100%|██████████| 600/600 [04:04<00:00,  2.45it/s]


Epoch: 0002 cost = 0.265858291


...

Epoch: 0003 cost = 0.254805452


100%|██████████| 600/600 [06:21<00:00,  1.57it/s]


Epoch: 0004 cost = 0.251252859


100%|██████████| 600/600 [07:30<00:00,  1.33it/s]

Epoch: 0005 cost = 0.247045677
Learning Finished!


# 예측하기

In [16]:
input_shape = X_test.shape[1] * X_test.shape[2]
X_test = X_test.reshape(-1, input_shape) 
y_test = to_categorical(y_test, nb_classes)

In [17]:
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print("Accuracy:", sess.run(accuracy, feed_dict={X: X_test, Y: y_test, keep_prob: 1}))

Accuracy: 0.9131
